WANT TO ADD A SUBURB/MUNICIPALITY TO THE ANALYZED TWEET FILE!!

## 1. Various libs and file location

In [131]:
import geopandas
from shapely.geometry import shape, Point, Polygon
import json

#CHANGE THESE VARIABLES!
bound_file = "D:\\MSDS\\2021 Sem 1\\Cloud Comp\\Ass2\\Saptial files\\sa3 bounds\\SA3_2016_AUST.shp"

twitter_file = 'analyzed_twitter.json'
outfile = 'analyzed_twitter2.json'

## 2. Processing the SA3 bounds file

In [88]:
'''
Reading the shape file and loading to a geopandas DF. Note that you need ALL the shape file extensions.
'''
#Load data into sa3_bounds
sa3_bound = geopandas.read_file(bound_file)


In [109]:
#Subset the states to only look for Vicrorian states
sa3_bound_vic = sa3_bound[sa3_bound['STE_NAME16']=='Victoria'].reset_index()

length_sa3_bound = sa3_bound_vic.shape[0]


In [141]:
'''
Function that returns the SA3 location of tweet given some point
'''
def sa3_bounds_given_point(point):
    location = 'none'
    sa3_code = 0
    # check each polygon to see if it contains the point
    for i in range(length_sa3_bound):
        feature = sa3_bound_vic.loc[i, 'geometry']
        if feature is not None:
            polygon = shape(feature)
            if polygon.contains(point):
                return sa3_bound_vic.loc[i, 'SA3_NAME16']                
    

## 3. Processing the twitter file

In [1]:
'''
Read twitter file
'''
#read tweet data
tt = open(twitter_file,'r',encoding='utf-8').read()
twitter_data = json.loads(tt)


In [91]:
def coordinate_of_tweet(sample_tweet):
    #Take the second point
    lon = sample_tweet['box coordinates'][0][1][0]
    lat = sample_tweet['box coordinates'][0][1][1]

    point = Point(lon, lat)

    return point

In [143]:
'''
Iterate through tweet this takse a while.
'''
updated_tweets = []

for tweet in twitter_data:
    #define the coordiate
    point = coordinate_of_tweet(tweet)
    #Search for coordinate
    sa3_location = sa3_bounds_given_point(point)
    #Put in dictionary, easier to append
    location_dict = {}
    location_dict['sa3_location'] = sa3_location
    
    #Update tweet
    tweet.update(location_dict)
    
    #put in updated_tweets
    updated_tweets.append(tweet)
    
    

## 3. Write back out as json file

In [144]:
with open(outfile,'w') as jsonFile:
    json.dump(updated_tweets, jsonFile)